## Importing Libraries

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import logging
from tqdm import tqdm
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


## Setting up data

In [4]:
import json
with open("/content/drive/MyDrive/TUDelft/NLP Group Project/data/programfc/programfc_preprocessed.json") as f:
  train_data = json.load(f)
len(train_data)

2605

# BM25

In [5]:
pip install rank_bm25

In [6]:
import re
from rank_bm25 import BM25Okapi

In [7]:
with open("/content/drive/MyDrive/TUDelft/NLP Group Project/bm25/corpus_evidence_unified.json") as f:
  document_corpus = json.load(f)

In [8]:
def preprocess_text(text):
	text = text.lower()
	text = re.sub(r'\s\.\.\.', '', text)
	text = re.sub(r'\.', '', text)
	text = text.split(" ")
	return text

In [9]:
corpus = list(document_corpus.values())
tokenized_corpus = []
for doc in corpus:
	tokenized_corpus.append(preprocess_text(doc))

bm25 = BM25Okapi(tokenized_corpus)

## ABHISHEKS CODE

In [10]:
def run_bm25(query, top_k):
  results = bm25.get_top_n(preprocess_text(query), corpus, n=top_k)
  return results

In [11]:
def do_bm25(data, save_path, pick_up_from=None):

  features = []
  start_index = 2000
  if pick_up_from:
      with open(pick_up_from, 'r') as file:
          features = json.load(file)
          start_index = start_index+len(features)  # Determine where to start processing
          print(f"Picking up from {start_index} claims")

  def save_progress(features):
        with open(save_path, 'w') as file:
            json.dump(features, file, indent=2)
        print("saved progress to ", save_path)

  # for i, example in enumerate(tqdm(data)):
  for i in tqdm(range(start_index, len(data)), desc="Processing Claims"):

    claim = data[i]['claim']
    q1 = data[i]['q1']
    q2 = data[i]['q2']
    q3 = data[i]['q3']
    results = []
    top_k = 3

    if q2 != "":
      top_k = 2
      results.extend(run_bm25(q2, top_k))

    results.extend(run_bm25(q1, top_k))
    results.extend(run_bm25(q3, top_k))

    evidences = ' '.join(results)
    feature_item = {
            "claim_evidence_string": f"[Claim]:{claim} [Evidences]:{evidences}",
            "idx": data[i]['idx'],
            "taxonomy_label": data[i]['taxonomy_label'],
            "label": data[i]['label']

        }
    features.append(feature_item)


    if (len(features) - start_index) % 10 == 0:
                save_progress(features)

  save_progress(features)


In [13]:
train_features = do_bm25(data=train_data,
                          save_path = '/content/drive/MyDrive/TUDelft/NLP Group Project/bm25_of_programfc/bm25_features_train_set.json'
                          # pick_up_from='None'
                        )

Processing Claims:   2%|▏         | 10/605 [01:11<1:09:13,  6.98s/it]

saved progress to  /content/drive/MyDrive/TUDelft/NLP Group Project/bm25_of_programfc/bm25_features_train_set.json


Processing Claims:   3%|▎         | 20/605 [02:24<1:08:25,  7.02s/it]

saved progress to  /content/drive/MyDrive/TUDelft/NLP Group Project/bm25_of_programfc/bm25_features_train_set.json


Processing Claims:   5%|▍         | 30/605 [03:21<1:02:59,  6.57s/it]

...

saved progress to  /content/drive/MyDrive/TUDelft/NLP Group Project/bm25_of_programfc/bm25_features_train_set.json


Processing Claims:  99%|█████████▉| 600/605 [1:13:02<00:41,  8.25s/it]

saved progress to  /content/drive/MyDrive/TUDelft/NLP Group Project/bm25_of_programfc/bm25_features_train_set.json


Processing Claims: 100%|██████████| 605/605 [1:13:40<00:00,  7.31s/it]

saved progress to  /content/drive/MyDrive/TUDelft/NLP Group Project/bm25_of_programfc/bm25_features_train_set.json
